## Align utterance in Memor to open subtitle parallel bitext

In [239]:
import json
from lxml import etree
import lxml
from tqdm import tqdm
import pickle as pkl
import re

#### Load MEMOR Dataset

In [2]:
# Load Memor Dataset
with open('memor/data.json') as f:
    data = json.load(f)

In [273]:
"""
1.Get utterances from a certain episode using episode_key
2.Split utterances into segments using puncutation: , . ?
"""
def get_segments_old(episode_key, all_data):
    # Gather utteranes the defined episode
    episode = []
    for x in all_data:
        if x.strip().split('_')[0]==episode_key:
            episode.extend(data[x]['sentences'])

    # Split utterances into segments
    temp = []
    for x in episode:
        temp.extend(x.strip().split('.'))
    episode = temp

    temp = []
    for x in episode:
        temp.extend(x.strip().split(','))
    episode = temp

    temp = []
    for x in episode:
        temp.extend(x.strip().split('?'))
    episode = temp

    temp = []
    for x in episode:
        temp.extend(x.strip().split('!'))
    episode = temp

    temp = []
    for x in episode:
        if len(x.strip().split(' ')) > 5:
            temp.append(x)
    episode = temp

    return episode

In [278]:
"""
1.Get utterances from a certain episode using episode_key
2.Split utterances into segments using puncutation: , . ?
"""
def get_segments(episode_key, all_data):
    # Gather utteranes the defined episode
    segments = []
    pattern = r'\.|\,|\?|\!|\;'
    for x in all_data:
        if x.strip().split('_')[0]==episode_key:
            # Collect Segments
            for utt in data[x]['sentences']:
                for item in re.split(pattern, utt):
                    tokens = item.strip().split(' ')
                    length = len(tokens)
                    num_iter = length // 6
                    for i in range(num_iter):
                        segments.append(" ".join(tokens[i*6: i*6+6]))
    return segments

In [222]:
episode = []
# Collect memor data from the same episode
for item in data:
    if item.strip().split('_')[0]=='S01E02' or item.strip().split('_')[0]=='S01E03' or item.strip().split('_')[0]=='S01E01':
        episode.append(data[item])

In [224]:
print(len(episode))

126


In [207]:
import re

pattern = r'\.|\,|\?|\!|\;'
text = "This is a, apple day. But I don't want it? anymore, how about! you?"
results = re.split(pattern, text)
print(results)

['This is a', ' apple day', " But I don't want it", ' anymore', ' how about', ' you', '']


In [236]:
# Try with Re in Segments
# Split sentence into small segments based on , and use segments to search the open subtitle corpus
segments = []
for x in episode:
    utts = x['sentences']
    for utt in utts:
        # print(utt)
        # print(re.split(pattern, utt))
        for item in re.split(pattern, utt):
            item_tokens = item.strip().split(' ')
            length = len(item_tokens)
            num_iter = length // 6
            # if length>=6:
            #     print(num_iter, length, item.strip())
            print(item)
            for i in range(num_iter):
                segments.append(" ".join(item_tokens[i*6: i*6+6]))
                # print(" ".join(item_tokens[i*6: i*6+6]))
            print("=="*50)


        # print('=='*50)
        # for sub_utt in utt.strip().split(','):
        #     sub_utt = sub_utt.strip().split('.')
        #     for item in sub_utt:
        #         temp = item.strip()
        #         if len(temp.split(' ')) > 6:
        #             print(temp)
        #             segments.append(temp)
        #         else:
        #             print('====', temp)
    # print('=='*50)

if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits
 if it's unobserved it will
 however
 if it's observed after it's left the plane but before it hits its target
 it will not have gone through both slits

agreed
 what's your point

there's no point
 i just think it's a good idea for a t-shirt

one across is aegean
 eight down is nabakov
 twenty-six across is mcm
 fourteen down is


 move your finger


 phylum
 which makes fourteen across port-au-prince
 see
 papa doc's capital idea
 that's port-au-prince
 haiti

no
 we are committing genetic fraud
 there's no guarantee that our sperm is going to generate high iq offspring
 think about that
 i have a sister with the same basic dna mix who hostesses at fuddruckers

sheldon
 this was your idea
 a little extra money to get fractional t1 bandwidth in the apartment

i know
 and i do yearn for faster downloads
 but there's some poor woman is going to pin her hopes on my s

In [238]:
print(len(segments))
print(segments)

355
['if a photon is directed through', 'a plane with two slits in', 'it and either slit is observed', 'it will not go through both', "if it's observed after it's left", 'the plane but before it hits', 'it will not have gone through', "i just think it's a good", "there's no guarantee that our sperm", 'is going to generate high iq', 'i have a sister with the', 'same basic dna mix who hostesses', 'a little extra money to get', 'fractional t1 bandwidth in the apartment', 'and i do yearn for faster', "but there's some poor woman is", 'going to pin her hopes on', 'what if she winds up with', "a toddler who doesn't know if", 'he should use an integral or', 'a differential to solve the area', "i'm sure she'll still love him", "i'm sure she'll still love him", 'what do you want to do', "i've never reneged on a proffer", 'if the height of a single', 'step is off by as little', 'i did a series of experiments', 'significant improvement over the old neighbour', 'two hundred pound transvestite with

In [54]:
# Split sentence into small segments based on , and use segments to search the open subtitle corpus
segments = []
for x in episode:
    utts = x['sentences']
    for utt in utts:
        for sub_utt in utt.strip().split(','):
            sub_utt = sub_utt.strip().split('.')
            for item in sub_utt:
                temp = item.strip()
                if len(temp.split(' ')) > 6:
                    print(temp)
                    segments.append(temp)
                else:
                    print('====', temp)
        # temp_1 = utt.strip().split(',')

        # for segment in temp:
        #     if len(segment.split(" "))<6:
        #         print('====', segment)
        #     else:
        #         print(segment)
        #         segments.append(segment)
    print('=='*50)

==== no
==== i haven't
==== 
==== get used to it
==== 
==== yeah
==== i probably won't
==== but
==== 
==== 
==== hey sheldon
==== 
==== hi
==== 
hey raj!  still not talking to me
==== huh?
==== don't take it personally
==== it's his pathology
==== he can't talk to women
==== 
==== he can't talk to attractive women
or in your case a cheesecake-scented goddess!
==== so
==== there's gonna be some furniture delivered?
==== oh no
==== let's assume that they can
==== lois lane is falling
accelerating at an initial rate of 32 feet per second per second
superman swoops down to save her by reaching out two arms of steel
==== miss lane
who is now travelling at approximately 120 miles per hour
==== hits them
and is immediately sliced into three equal pieces
==== 
==== are you listening to yourself
it is well established that superman's flight is a feat of strength
it is an extension of his ability to leap tall buildings
an ability he derives from earth's yellow sun
==== 
==== yeah
and you don't h

In [55]:
print(len(segments))

67


In [57]:
for x in segments:
    print(x)

hey raj!  still not talking to me
or in your case a cheesecake-scented goddess!
accelerating at an initial rate of 32 feet per second per second
superman swoops down to save her by reaching out two arms of steel
who is now travelling at approximately 120 miles per hour
and is immediately sliced into three equal pieces
it is well established that superman's flight is a feat of strength
it is an extension of his ability to leap tall buildings
an ability he derives from earth's yellow sun
and you don't have a problem with that
a combination of the moon's solar reflection and the energy storage capacity of kryptonian skin cells
a combination of the moon's solar reflection and the energy storage capacity of kryptonian skin cells
i'm just going to go wash up
i have 26 hundred comic books in there
i challenge you to find a single reference to kryptonian skin cells
i'm just going to go wash up
i have 26 hundred comic books in there
i challenge you to find a single reference to kryptonian skin 

#### Load Open Subtitle BiText Corpus

In [3]:
# Load Open Subtitle Bitext
xml_file = etree.parse('../open_subtitle/en-zh_cn.tmx')

In [36]:
# Collect subtitles
en_subtitle = []
zh_subtitle = []
root = xml_file.getroot()
count = 0
for node in tqdm(root.iter()):
    if node.tag=='seg':
        if count % 2 ==0:
            en_subtitle.append(node.text)
        else:
            zh_subtitle.append(node.text)
        count += 1

49845028it [00:35, 1398557.88it/s]


In [39]:
# Check the processed subtitles
print(len(en_subtitle))
print(len(zh_subtitle))
for i in range(10):
    print(en_subtitle[i])
    print(zh_subtitle[i])
    print('=='*50)
for i in range(10):
    print(en_subtitle[-i])
    print(zh_subtitle[-i])
    print('=='*50)

Ah, this is greasy. I want to eat kimchee.
好想要吃泡菜
Is Chae Yoon's coordinator in here?
崔允的造型师在吗
Excuse me, aren't you Chae Yoon's coordinator?
请问一下 你是不是崔允的造型师
Yes. Me?
我吗
-Chae Yoon is done singing.
崔允已经唱完了
This lady right next to me... everyone knows who she is right?
这位小姐是谁 我想大家都知道吧
She is my forever idol...she is my rival whom I can never win against, she is the one who always makes me happy, and she is my mother.
她是我永远的偶像 也是我无法击倒的对手 她是令我怦然心动的恋人
Everyone should remember my mom as one of the greatest movie stars, but instead of becoming an actress, she wanted to become a singer just like me.
也许在各位的记忆里 我母亲 她是曾经红极一时的女演员
When I was stressed and exhausted, she'd always sing me a song while having me on her lap.
而是像我一样当歌手 当我感到累还有疲倦的时候 每一次她都让我
Today, I would like to share that moment with everyone.
然后为我唱一首歌 今天我想要 特别和各位
Ah, this is greasy. I want to eat kimchee.
好想要吃泡菜
Subtitle translation by Brian Athey
字幕翻译：
MY LADIES BY SHIN MICHIMA
（我的女人们）
It's because... that's the way I want it.
这是我自己的

In [67]:
pkl.dump(en_subtitle, open('en_subtitles.pkl', 'wb'))
pkl.dump(zh_subtitle, open('zh_subtitles.pkl', 'wb'))

#### Align the MRMOR Corpus with the Open Subtitle Corpus

In [87]:
# Load Open Subtitle
with open('en_subtitles.pkl', 'rb') as f:
    en_subtitle = pkl.load(f)
with open('zh_subtitles.pkl', 'rb') as f:
    zh_subtitle = pkl.load(f)

In [92]:
# Load MEMOR Dataset
with open('memor/data.json') as f:
    data = json.load(f)

In [282]:
episode_keys = set()
for x in data:
    episode_keys.add(x.strip().split('_')[0])
print(len(episode_keys))
print(tuple(episode_keys))

204
('S03E06', 'S03E19', 'S09E02', 'S09E09', 'S05E22', 'S07E18', 'S09E07', 'S02E19', 'S08E23', 'S07E11', 'S06E06', 'S07E03', 'S02E13', 'S04E17', 'S04E01', 'S01E09', 'S08E19', 'S03E10', 'S07E22', 'S01E12', 'S08E17', 'S03E21', 'S03E23', 'S09E20', 'S02E08', 'S08E09', 'S09E11', 'S06E05', 'S03E04', 'S02E23', 'S05E12', 'S08E14', 'S08E07', 'S07E17', 'S01E10', 'S04E23', 'S03E07', 'S01E06', 'S06E15', 'S06E14', 'S01E01', 'S03E09', 'S06E09', 'S08E15', 'S01E03', 'S07E19', 'S02E17', 'S03E20', 'S05E02', 'S02E10', 'S09E14', 'S03E18', 'S05E01', 'S06E19', 'S09E10', 'S01E05', 'S09E16', 'S05E19', 'S03E22', 'S05E03', 'S05E16', 'S02E03', 'S06E03', 'S04E08', 'S02E02', 'S02E07', 'S07E08', 'S04E15', 'S06E04', 'S01E08', 'S04E07', 'S03E02', 'S07E10', 'S06E08', 'S04E09', 'S05E07', 'S01E16', 'S07E06', 'S04E18', 'S08E08', 'S06E23', 'S07E24', 'S05E17', 'S06E11', 'S08E03', 'S07E21', 'S09E03', 'S02E05', 'S04E11', 'S03E12', 'S07E01', 'S05E18', 'S08E04', 'S05E11', 'S01E07', 'S03E01', 'S02E11', 'S02E16', 'S08E13', 'S06E

In [291]:
episode_indexs = {}
# for item in tqdm(tuple(episode_keys)[:2]):
for item in tqdm(['S01E09', 'S01E16']):
    temp = []
    print(item)
    segments = get_segments(item, data)
    for segment in tqdm(segments):
        for i, subtitle in enumerate(en_subtitle):
            if segment in subtitle:
                temp.append([i, en_subtitle[i], zh_subtitle[i]])
    episode_indexs[item] = temp
# pkl.dump(episode_indexs, open('episode_indexs.pkl', 'wb'))

  0%|          | 0/2 [00:00<?, ?it/s]

S01E09



 50%|█████     | 1/2 [01:06<01:06, 66.52s/it]

S01E16



100%|██████████| 2/2 [02:23<00:00, 71.87s/it]


In [292]:
# print(episode_indexs)
for x in episode_indexs:
    print(x)
    temp = episode_indexs[x]
    for item in temp:
        print(item)
    # print(episode_indexs[x])
    print('=='*50)

S01E09
[4701261, 'Gentlemen, I am now about to send a signal from this laptop through our local ISP racing down fibre optic cable at the speed of light to San Fransisco, bouncing off a satellite in geosynchronous orbit to Lisbon, Portugal,', '同志们，我现在就要由笔记本经由本地ISP发送信号了 信号将以光速经由纤维光学电缆传递到San Francisco 经过葡萄牙里斯本上空轨道上的卫星']
[4701261, 'Gentlemen, I am now about to send a signal from this laptop through our local ISP racing down fibre optic cable at the speed of light to San Fransisco, bouncing off a satellite in geosynchronous orbit to Lisbon, Portugal,', '同志们，我现在就要由笔记本经由本地ISP发送信号了 信号将以光速经由纤维光学电缆传递到San Francisco 经过葡萄牙里斯本上空轨道上的卫星']
[4701262, 'where the data packets will be headed off to submerge transatlantic cables, terminating in Halifax, Nova Scotia and transfered across the continent via microwave relays back to our ISP', '然后数据打包穿过大西洋的海底电缆 终止于Nova Scotia省的Halifax市（位于加拿大，译者注） 然后经过大陆上的微波遥感传回我们的ISP']
[4701286, 'Someone in Sichuan province, China, is using his computer to turn our lights on and

In [59]:
for segment in segments:
    for i, subtitle in enumerate(en_subtitle):
        if segment in subtitle:
            print("Index:", i)
            print("Target_en:", segment)
            print("Target_zh:", zh_subtitle[i])
            print("Context:")
            for b in range(3):
                print(en_subtitle[i+2-b], zh_subtitle[i+2-b])
            print('=='*50)

Index: 4236022
Target_en: and is immediately sliced into three equal pieces
Target_zh: 莱恩小姐此时大约速度在120英里/时 猛撞上超人的手臂后 她会马上被切成三等分
Context:
In what space, sir? In what space? 哪还有时间 先生?
Unless Superman matches her speed and decelerates. 除非超人赶上她的速度并减速
Miss Lane, who is now traveling at approximately 120 miles an hour, hits them and is immediately sliced into three equal pieces. 莱恩小姐此时大约速度在120英里/时 猛撞上超人的手臂后 她会马上被切成三等分
Index: 4236085
Target_en: because the evidence suggests the coffee table's having a tiny garage sale
Target_zh: 现在 我只是假设这是沙发 因为证据显示 咖啡桌上正进行一场小型旧货出售
Context:
No. 没有想过
Did it occur to you that not everyone has the compulsive need to sort organize and label the entire world around them? 你有没想过 并不是每个人都觉得必须要... 归类 整理 并将一切事物都贴上标签?
I'm just inferring that this as a couch because the evidence suggests the coffee table's having a tiny garage sale. 现在 我只是假设这是沙发 因为证据显示 咖啡桌上正进行一场小型旧货出售
Index: 3498567
Target_en: i just want you to know that
Target_zh: 大卫 我只要你知道我要走了
Context:
i told you David, 

In [60]:
print(en_subtitle[4236022], zh_subtitle[4236022])

Miss Lane, who is now traveling at approximately 120 miles an hour, hits them and is immediately sliced into three equal pieces. 莱恩小姐此时大约速度在120英里/时 猛撞上超人的手臂后 她会马上被切成三等分


In [63]:
b = 300
for i in range(b):
    print(en_subtitle[4236022-b+i], zh_subtitle[4236022-b+i])
for i in range(b):
    print(en_subtitle[4236022+i], zh_subtitle[4236022+i])

- He had a brand scorched on his hand... －他胳膊上有烙印 －他说出了发送机的位置
He told my people where the transmitter is, and he gave the card with the 1st part of the code. 还交出了密码第一部分的卡片
The Norwegian guy wasn't so compliant, 在挪威遇到的人不善言谈
- but we only needed his portion of the code. - The brand! －但我们只需要他的数字 －烙印
Now only a trifle is left - the 3rd portion of the code. 现在我就缺一小部分 密码第三部分， 但是我不知道该去哪里找
- Me either. - You're wrong. －我也是 －你错了
I'll prove you you're wrong. Just come here. Alone. 我向你证明， 如果你到我这里来，一个人
Think about it. You guys don't know the transmitter's location 好好想想吧，我知道发送机在哪里， 而你们，不知道
- and the code's two parts. －我知道密码的前两部分，而你们...
- We know them. ... －我们也知道
- You're bluffing. - No. －你虚张声势 －没有
Tell me, ten. 你能说出来吗？
Eight-six-six. 8－6－6...
Now you. ... 现在你说
Seven-eight-four-tree. 7－8－4－3
Great, what now? 很好，接下来那？
We should become partners, for some time, to find the missing part of the code together. 呃，我们应该暂时结成同盟 找到我们还没有的那部分密码
- And what ten? －然后呢？
- Then I'll kill you, or you will kill me, time

In [120]:
with open('episode_indexs.pkl', 'rb') as f:
    results = pkl.load(f)

In [132]:
# Look alignment for each episode
for item in results:
    # print(item, len(results[item]))
    for x in results[item]:
        if len(results[item]) < 6:
            print(item, len(results[item]))
            print(x)
    print('=='*50)

S08E07 4
[2102125, 'this day just keeps getting better', 'Gosh, this-this day just keeps getting better and better.', '你们的麻烦大了 越来越精采了']
S08E07 4
[5404299, 'this day just keeps getting better', 'Oh, this day just keeps getting better.', '看来日子越来越好过了']
S08E07 4
[8057166, 'this day just keeps getting better', 'Well, this day just keeps getting better.', '日子越来越好了']
S08E07 4
[9565302, 'this day just keeps getting better', 'Oh, this day just keeps getting better.', '哦 今天发生的事越来越好玩了 Oh, this day just keeps getting better.']
S09E18 2
[1077833, 'get a load of this guy', "Biff, get a load of this guy's life preserver.", '嘿！ 你看，他穿救生衣']
S09E18 2
[3402051, 'get a load of this guy', 'Hey, get a load of this guy.', '你看看这家伙']
S03E06 5
[5444136, "you're not still grinding on the kite thing", "Come on, you're not still grinding on the kite thing, are you?", '你还耿耿于怀风筝的事啊?']
S03E06 5
[8439229, 'what was i supposed to do', 'what was i supposed to do?', '那我该怎么办？']
S03E06 5
[5444244, "that's a highlight from t

In [156]:
# Check the alignment using indexs
target_index_dict = {}

for item in results:
    print(item)
    dic = {}
    for x in results[item]:
        dic[x[0]] = x[1:]
    idxs = sorted(list(dic.keys()))
    print(idxs)
    gaps = []
    for i in range(len(idxs)-1):
        gaps.append(idxs[i+1]-idxs[i])
    print(gaps)
    for i in range(len(gaps)-3):
        if max(gaps[i: i+3]) < 1000:
            print("Target Indexs:", idxs[i: i+3])
            target_index_dict[item] = idxs[i: i+3]
            break
    print('=='*50)

S08E01
[91843, 434520, 485314, 1142809, 1169122, 1227545, 1244666, 1579247, 1657495, 1692143, 1918042, 2200592, 2339078, 2441820, 3331859, 3435848, 3526212, 3533199, 3542740, 3955086, 3988717, 4058810, 4355531, 4372232, 4438321, 4466366, 4518857, 4625619, 4952412, 5182467, 5282382, 5286887, 6098614, 6109006, 6121257, 6306820, 6621804, 6832430, 7033395, 7063953, 7086134, 7088975, 7214259, 7250242, 7513499, 7785060, 7930687, 8033779, 8480472, 8703063, 8905776, 8905795, 8905900, 8905950, 8905951, 8905952, 8905976, 8905997, 8906053, 8906066, 8906072, 8980247, 9057337, 9111948, 9404988, 9520474, 9569269, 9904345]
[342677, 50794, 657495, 26313, 58423, 17121, 334581, 78248, 34648, 225899, 282550, 138486, 102742, 890039, 103989, 90364, 6987, 9541, 412346, 33631, 70093, 296721, 16701, 66089, 28045, 52491, 106762, 326793, 230055, 99915, 4505, 811727, 10392, 12251, 185563, 314984, 210626, 200965, 30558, 22181, 2841, 125284, 35983, 263257, 271561, 145627, 103092, 446693, 222591, 202713, 19, 105, 5

In [158]:
for item in target_index_dict:
    print(item, target_index_dict[item])

S08E01 [8905776, 8905795, 8905900]
S02E04 [4799447, 4799458, 4799601]
S02E06 [4799982, 4800063, 4800101]
S08E05 [9003533, 9003536, 9003611]
S05E12 [7035657, 7035734, 7035735]
S05E13 [7037231, 7037254, 7037339]
S05E10 [6585550, 6585573, 6585623]
S05E11 [6594474, 6594487, 6594540]
S05E16 [7086436, 7086485, 7086490]
S05E17 [7092702, 7092732, 7092740]
S05E14 [7063772, 7063841, 7063848]
S05E15 [7070709, 7070779, 7070780]
S05E18 [7111270, 7111284, 7111296]
S05E19 [7137372, 7137388, 7137413]
S02E22 [5256711, 5256721, 5256751]
S02E23 [5377378, 5377393, 5377414]
S02E20 [5256126, 5256254, 5256300]
S02E21 [5256438, 5256441, 5256475]
S09E16 [9827807, 9827830, 9827867]
S09E14 [9825932, 9826013, 9826014]
S09E15 [9826977, 9826990, 9827015]
S09E12 [9823787, 9823788, 9823805]
S09E13 [9823365, 9823570, 9823615]
S09E10 [9575788, 9575829, 9575938]
S09E11 [9576136, 9576203, 9576259]
S06E24 [8058528, 8058544, 8058684]
S03E06 [5444136, 5444154, 5444174]
S05E01 [6513597, 6513613, 6513649]
S05E02 [6542840, 654

In [171]:
season_dict = {}
for item in target_index_dict:
    temp = item.strip().split('E')
    season_key = temp[0]
    episode_key = temp[1]
    if season_key not in season_dict:
        season_dict[season_key] = {}
    else:
        season_dict[season_key][episode_key] = target_index_dict[item]
            # .append({episode_key: target_index_dict[item]})

In [172]:
print(season_dict)

{'S08': {'05': [9003533, 9003536, 9003611], '08': [9007095, 9007255, 9007257], '09': [9007465, 9007501, 9007503], '02': [8999107, 8999108, 8999169], '03': [9002858, 9002904, 9002972], '06': [9006724, 9006727, 9006826], '04': [9003176, 9003179, 9003225], '19': [9381497, 9381513, 9381529], '18': [319626, 320615, 320618], '13': [9379758, 9379772, 9379788], '12': [9368331, 9368398, 9368423], '15': [9380083, 9380113, 9380147], '17': [9380791, 9380866, 9380903], '16': [9380426, 9380429, 9380448], '24': [9383227, 9383296, 9383299], '20': [9381885, 9381894, 9381899], '21': [9382241, 9382324, 9382325], '22': [9382624, 9382697, 9382744], '23': [9382980, 9382986, 9383082], '11': [9008032, 9008064, 9008134], '10': [9007795, 9007797, 9007821]}, 'S02': {'06': [4799982, 4800063, 4800101], '22': [5256711, 5256721, 5256751], '23': [5377378, 5377393, 5377414], '20': [5256126, 5256254, 5256300], '21': [5256438, 5256441, 5256475], '08': [4800627, 4800711, 4800800], '09': [4800880, 4800897, 4800917], '02':

In [179]:
for item in sorted(season_dict.keys()):
    episodes = []
    print(item)
    for x in sorted(season_dict[item].keys()):
        print(x, season_dict[item][x])
    # print(sorted(season_dict[item].keys()))
    # print(season_dict[item])
    print("=="*50)

S01
01 [4424331, 4424416, 4424418]
02 [4236022, 4236085, 4236108]
03 [4236332, 4236344, 4236374]
04 [4236606, 4236691, 4236776]
05 [4236908, 4237012, 4237044]
06 [4237202, 4237286, 4237376]
07 [4245874, 4245883, 4245884]
08 [4244392, 4244471, 4244472]
10 [4700111, 4700162, 4700166]
11 [4700365, 4700385, 4700528]
12 [4700628, 4700651, 4700709]
13 [4701003, 4701030, 4701067]
S02
01 [4813146, 4813154, 4813168]
02 [4798782, 4798802, 4798842]
03 [4799123, 4799222, 4799269]
06 [4799982, 4800063, 4800101]
07 [4800351, 4800363, 4800407]
08 [4800627, 4800711, 4800800]
09 [4800880, 4800897, 4800917]
12 [5253724, 5253750, 5253766]
13 [5254034, 5254075, 5254077]
14 [5254389, 5254393, 5254402]
15 [5254719, 5254748, 5254775]
16 [5254973, 5254981, 5255036]
17 [5255278, 5255298, 5255388]
18 [5255563, 5255652, 5255704]
19 [5255854, 5256053, 5256057]
20 [5256126, 5256254, 5256300]
21 [5256438, 5256441, 5256475]
22 [5256711, 5256721, 5256751]
23 [5377378, 5377393, 5377414]
S03
01 [5422087, 5422094, 54220

In [190]:
base = 4795131
for i in range(20):
    print(en_subtitle[base-20+i], zh_subtitle[base-20+i])
print("======", en_subtitle[base], zh_subtitle[base])
for i in range(20):
    print(en_subtitle[base+i], zh_subtitle[base+i])

Tell Fischer he woke up. 告诉Fischer 他醒了
What is... what is this place? 这里... 这里是什么地方?
Hey, what are you doing? What are you... 嘿 你在做什么?
No, no, no! 你在做...
No, wait, no... 不 不 不!
Please! Please, please, please! 不 等一下 不 求求你 别 别 别这样!
Yes, ideal. 对 非常好
Walter, you're not gonna accidentally fry One of those pigeons? Walter 你不会意外地 烤了这些鸽子吧?
Stranger things have happened. That's his motto. 更奇怪的事情也发生过
As I said, 这是他的格言
Pigeons contain traces of magnetite in their beaks. 像我说的 鸽子的喙里有一些磁铁矿
Like hundreds of tiny Compass needles. 就像数以百计的小指南针
The tesla coils will create An artificial electromagnetic field, 特斯拉线圈会产生人造电磁场
Which I have already matched With Mister... 我已经把它调整成了那位 嗯...
Uh, whatever's unique signature, And will now imprint onto the pigeon. 随便什么名字的先生的独特的标记 而那会在鸽子上留下印记
So we set all of these birds free and they... 所以我们放了这些鸽子 它们会?
Fly to Joseph Meegar. 会飞到Joseph Meegar那儿
Yeah, I know. Me too. I'll believe it when I see it. 我知道 我也要亲眼见到才会相信
Step back. Work to be done. 退后 要开工了
GPS chip's ready? Ye